In [1]:
!nvidia-smi

Wed Oct 28 07:59:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%cd drive/My\ Drive/samsung_card/data/
!ls

/content/drive/My Drive/samsung_card/data
cst_feat_feb_quiz.csv  mrc_info.csv  train.csv
cst_feat_jan.csv       quiz.csv      val.csv


In [3]:
import pandas as pd
import numpy as np

In [4]:
df_train = pd.read_csv('train.csv')
df_val = pd.read_csv('val.csv')

In [5]:
df_y_value = pd.concat([df_train, df_val])
df_y_value.set_index('cst_id_di', inplace=True)

In [6]:
df_y_value.head()

,MRC_ID_DI
cst_id_di,
90000000003,0
90000000006,0
90000000007,0
90000000008,0
90000000009,0


In [7]:
df_jan = pd.read_csv('cst_feat_jan.csv')
df_jan.set_index('cst_id_di', inplace=True)

In [8]:
df_jan

,VAR002,VAR003,VAR004,VAR005,VAR006,VAR007,VAR008,VAR009,VAR010,VAR011,VAR012,VAR013,VAR014,VAR015,VAR016,VAR017,VAR018,VAR019,VAR020,VAR021,VAR022,VAR023,VAR024,VAR025,VAR026,VAR027,VAR028,VAR029,VAR030,VAR031,VAR032,VAR033,VAR034,VAR035,VAR036,VAR037,VAR038,VAR039,VAR040,VAR041,...,VAR188,VAR189,VAR190,VAR191,VAR192,VAR193,VAR194,VAR195,VAR196,VAR197,VAR198,VAR199,VAR200,VAR201,VAR202,VAR203,VAR204,VAR205,VAR206,VAR207,VAR208,VAR209,VAR210,VAR211,VAR212,VAR213,VAR214,VAR215,VAR216,VAR217,VAR218,VAR219,VAR220,VAR221,VAR222,VAR223,VAR224,VAR225,VAR226,VAR227
cst_id_di,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
90000000003,-0.02438,0.5574,0.03330,-0.16819,0.47308,0,0.4832,0.22558,0.7030,0.52756,0.37048,-0.46270,-0.51311,0,0.0053,0.0457,0,-0.47469,-0.05412,0.35447,0.02470,0.08281,0.15881,0.31802,0,0.49211,0.42831,-0.29062,-0.14369,0.0301,-0.48468,0.06394,0.37780,-0.38244,0.0029,-0.25822,0.44000,0.43275,-0.04608,-0.14626,...,0.21727,0.43217,0.08468,-0.40895,0.1794,-0.09317,-0.08511,-0.08127,-0.59618,0.0054,-0.23635,0,0.17488,-0.16801,-0.45058,0.68462,0.69091,0.25896,-0.08855,-0.17158,0,-0.26256,-0.24129,0.05623,-0.10093,-0.00042,0.23027,0.26068,-0.35145,0.17076,0.46899,-0.37015,-0.08146,-0.14729,-0.48325,-0.13221,-0.07368,-0.00088,-0.27906,0.24657
90000000006,0.01635,0.9840,0.33343,-0.35664,0.31725,1,0.3642,0.61984,0.0307,0.45824,0.34812,-0.34764,-0.01273,1,0.0042,0.0198,0,-0.26118,-0.19758,0.59550,0.12663,-0.17034,-0.45976,0.39790,0,0.08345,0.03025,-0.37373,-0.27354,0.0490,-0.21326,-0.13410,0.45987,-0.45736,0.6432,-0.46337,0.05895,0.66428,-0.07391,-0.44666,...,-0.00779,-0.07561,0.24172,0.07813,0.6428,-0.29811,-0.25871,-0.26234,-0.27024,0.0048,-0.37216,0,0.31705,-0.27720,-0.36614,0.66847,0.63441,-0.25729,-0.28089,-0.35974,0,-0.43106,-0.14855,-0.37842,-0.20628,-0.30776,0.26571,0.57852,-0.43703,-0.45285,0.61719,-0.57482,-0.30712,-0.31664,-0.69605,-0.35557,-0.12839,-0.24843,-0.39112,0.42928
90000000007,-0.02634,0.1279,-0.11887,-0.04039,0.60425,0,0.2180,-0.01968,0.5214,0.44441,0.58878,-0.33039,-0.33452,1,0.0023,0.0085,0,-0.22726,0.00189,0.16102,0.25061,0.02793,0.24411,-0.09407,0,0.36461,-0.09348,-0.00705,-0.04356,0.6081,-0.27057,0.42279,0.07238,-0.07926,0.2426,-0.02265,0.06478,0.11117,-0.08222,-0.06549,...,0.02196,0.11668,0.18771,-0.02905,0.2835,-0.00773,-0.00116,0.00468,-0.25456,0.0051,0.10580,0,-0.23435,-0.08062,-0.29454,0.28955,0.42931,0.07685,-0.01675,-0.02860,0,0.19778,-0.00939,-0.00897,0.00482,0.01496,0.29840,-0.17636,-0.45322,-0.01951,-0.04259,-0.00811,-0.00955,-0.04018,0.01891,-0.04929,-0.00756,0.04412,-0.16766,0.36213
90000000008,0.06713,0.4197,0.06721,-0.10704,0.49907,1,0.7085,0.19513,0.1202,0.70449,0.51032,-0.50266,-0.43218,1,0.0011,0.7353,0,-0.47922,-0.02628,0.35980,0.12944,-0.07166,0.01994,0.22894,0,0.43761,0.29530,-0.08881,-0.06188,0.9999,-0.45538,-0.03847,0.34738,-0.42881,0.2862,-0.12805,0.31678,0.58809,-0.07443,-0.26650,...,-0.05402,0.35317,0.24205,-0.28790,0.6588,-0.09018,-0.07387,-0.05642,-0.56804,0.0023,-0.28387,0,0.19651,-0.08654,-0.49406,0.77439,0.65251,-0.15073,-0.08173,-0.12238,0,-0.07976,0.04392,-0.17632,0.09252,-0.10762,0.23352,0.35249,-0.48637,-0.01310,0.37356,-0.33001,-0.05702,-0.09469,-0.45637,-0.04424,0.04400,-0.06770,-0.31665,0.40457
90000000009,0.00197,0.7817,0.49927,-0.33174,0.39280,1,0.9822,0.59993,0.0519,0.43291,0.23227,-0.40569,-0.52101,0,0.7189,0.2720,0,-0.44182,-0.19742,0.53594,-0.08657,-0.17877,0.18997,0.54729,0,0.52164,0.45655,-0.36219,-0.24241,0.1302,-0.49630,-0.19536,0.55551,-0.43173,0.1273,-0.46147,0.46253,0.42513,-0.02572,-0.16340,...,-0.13429,0.31715,-0.03167,-0.45375,0.8576,-0.26714,-0.23458,-0.23356,-0.54685,0.0021,-0.29772,0,0.30964,-0.25563,-0.37721,0.73192,0.69240,-0.28603,-0.25482,-0.34293,0,-0.45254,-0.21414,-0.20891,-0.21710,-0.24814,0.28316,0.58618,-0.25255,0.16567,0.67472,-0.57923,-0.26024,-0.29494,-0.66088,-0.27825,-0.11877,-0.24520,-0.37254,0.19273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [9]:
df_merged = pd.merge(df_y_value, df_jan, left_index=True, right_index=True, how='left')

In [10]:
df_merged.head()

,MRC_ID_DI,VAR002,VAR003,VAR004,VAR005,VAR006,VAR007,VAR008,VAR009,VAR010,VAR011,VAR012,VAR013,VAR014,VAR015,VAR016,VAR017,VAR018,VAR019,VAR020,VAR021,VAR022,VAR023,VAR024,VAR025,VAR026,VAR027,VAR028,VAR029,VAR030,VAR031,VAR032,VAR033,VAR034,VAR035,VAR036,VAR037,VAR038,VAR039,VAR040,...,VAR188,VAR189,VAR190,VAR191,VAR192,VAR193,VAR194,VAR195,VAR196,VAR197,VAR198,VAR199,VAR200,VAR201,VAR202,VAR203,VAR204,VAR205,VAR206,VAR207,VAR208,VAR209,VAR210,VAR211,VAR212,VAR213,VAR214,VAR215,VAR216,VAR217,VAR218,VAR219,VAR220,VAR221,VAR222,VAR223,VAR224,VAR225,VAR226,VAR227
cst_id_di,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
90000000003,0,-0.02438,0.5574,0.03330,-0.16819,0.47308,0,0.4832,0.22558,0.7030,0.52756,0.37048,-0.46270,-0.51311,0,0.0053,0.0457,0,-0.47469,-0.05412,0.35447,0.02470,0.08281,0.15881,0.31802,0,0.49211,0.42831,-0.29062,-0.14369,0.0301,-0.48468,0.06394,0.37780,-0.38244,0.0029,-0.25822,0.44000,0.43275,-0.04608,...,0.21727,0.43217,0.08468,-0.40895,0.1794,-0.09317,-0.08511,-0.08127,-0.59618,0.0054,-0.23635,0,0.17488,-0.16801,-0.45058,0.68462,0.69091,0.25896,-0.08855,-0.17158,0,-0.26256,-0.24129,0.05623,-0.10093,-0.00042,0.23027,0.26068,-0.35145,0.17076,0.46899,-0.37015,-0.08146,-0.14729,-0.48325,-0.13221,-0.07368,-0.00088,-0.27906,0.24657
90000000006,0,0.01635,0.9840,0.33343,-0.35664,0.31725,1,0.3642,0.61984,0.0307,0.45824,0.34812,-0.34764,-0.01273,1,0.0042,0.0198,0,-0.26118,-0.19758,0.59550,0.12663,-0.17034,-0.45976,0.39790,0,0.08345,0.03025,-0.37373,-0.27354,0.0490,-0.21326,-0.13410,0.45987,-0.45736,0.6432,-0.46337,0.05895,0.66428,-0.07391,...,-0.00779,-0.07561,0.24172,0.07813,0.6428,-0.29811,-0.25871,-0.26234,-0.27024,0.0048,-0.37216,0,0.31705,-0.27720,-0.36614,0.66847,0.63441,-0.25729,-0.28089,-0.35974,0,-0.43106,-0.14855,-0.37842,-0.20628,-0.30776,0.26571,0.57852,-0.43703,-0.45285,0.61719,-0.57482,-0.30712,-0.31664,-0.69605,-0.35557,-0.12839,-0.24843,-0.39112,0.42928
90000000007,0,-0.02634,0.1279,-0.11887,-0.04039,0.60425,0,0.2180,-0.01968,0.5214,0.44441,0.58878,-0.33039,-0.33452,1,0.0023,0.0085,0,-0.22726,0.00189,0.16102,0.25061,0.02793,0.24411,-0.09407,0,0.36461,-0.09348,-0.00705,-0.04356,0.6081,-0.27057,0.42279,0.07238,-0.07926,0.2426,-0.02265,0.06478,0.11117,-0.08222,...,0.02196,0.11668,0.18771,-0.02905,0.2835,-0.00773,-0.00116,0.00468,-0.25456,0.0051,0.10580,0,-0.23435,-0.08062,-0.29454,0.28955,0.42931,0.07685,-0.01675,-0.02860,0,0.19778,-0.00939,-0.00897,0.00482,0.01496,0.29840,-0.17636,-0.45322,-0.01951,-0.04259,-0.00811,-0.00955,-0.04018,0.01891,-0.04929,-0.00756,0.04412,-0.16766,0.36213
90000000008,0,0.06713,0.4197,0.06721,-0.10704,0.49907,1,0.7085,0.19513,0.1202,0.70449,0.51032,-0.50266,-0.43218,1,0.0011,0.7353,0,-0.47922,-0.02628,0.35980,0.12944,-0.07166,0.01994,0.22894,0,0.43761,0.29530,-0.08881,-0.06188,0.9999,-0.45538,-0.03847,0.34738,-0.42881,0.2862,-0.12805,0.31678,0.58809,-0.07443,...,-0.05402,0.35317,0.24205,-0.28790,0.6588,-0.09018,-0.07387,-0.05642,-0.56804,0.0023,-0.28387,0,0.19651,-0.08654,-0.49406,0.77439,0.65251,-0.15073,-0.08173,-0.12238,0,-0.07976,0.04392,-0.17632,0.09252,-0.10762,0.23352,0.35249,-0.48637,-0.01310,0.37356,-0.33001,-0.05702,-0.09469,-0.45637,-0.04424,0.04400,-0.06770,-0.31665,0.40457
90000000009,0,0.00197,0.7817,0.49927,-0.33174,0.39280,1,0.9822,0.59993,0.0519,0.43291,0.23227,-0.40569,-0.52101,0,0.7189,0.2720,0,-0.44182,-0.19742,0.53594,-0.08657,-0.17877,0.18997,0.54729,0,0.52164,0.45655,-0.36219,-0.24241,0.1302,-0.49630,-0.19536,0.55551,-0.43173,0.1273,-0.46147,0.46253,0.42513,-0.02572,...,-0.13429,0.31715,-0.03167,-0.45375,0.8576,-0.26714,-0.23458,-0.23356,-0.54685,0.0021,-0.29772,0,0.30964,-0.25563,-0.37721,0.73192,0.69240,-0.28603,-0.25482,-0.34293,0,-0.45254,-0.21414,-0.20891,-0.21710,-0.24814,0.28316,0.58618,-0.25255,0.16567,0.67472,-0.57923,-0.26024,-0.29494,-0.66088,-0.27825,-0.11877,-0.24520,-0.37254,0.19273


In [11]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, label_binarize
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
xgb_default = XGBClassifier(kvargs={'tree_method':'gpu_hist'})

In [12]:
def multiclass_roc_auc_score(y_test, y_pred, average=None):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    return roc_auc_score(y_test, y_pred, average=average)

In [13]:
X = df_merged.iloc[:, 1:].values
y = df_merged.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)


In [ ]:
#%time xgb_default.fit(X_train, y_train)

In [28]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier()

In [29]:
%time lgbm.fit(X_train, y_train)

CPU times: user 8min 24s, sys: 811 ms, total: 8min 25s
Wall time: 4min 17s


LGBMClassifier()

In [30]:
y_pred = lgbm.predict_proba(X_test)

In [40]:
print(multiclass_roc_auc_score(y_test, y_pred), multiclass_roc_auc_score(y_test, y_pred, average='macro'))

[0.88401776 0.84583031 0.94750301 0.83326577 0.86097839 0.95011489
 0.80702925 0.85025725 0.83452586 0.90639002 0.94921571] 0.8790116562067453


In [51]:
def LIFT20 (original_df, predict_Df):
    total_data = len(original_df)
    LIFT_value = []
    for classes in range(11):
        df = predict_df.sort_values(by=['가맹점%s'%classes], ascending=False)
        df = df[:int(len(predict_df)*0.2)]
        
        denominator = len(original_df[original_df['MRC_ID_DI']==int(classes)])/total_data
        numerator = len(df[df['y_test'] == classes])/int(len(predict_df)*0.2)
        LIFT_value.append(numerator/denominator)
        
    return LIFT_value, np.mean(LIFT_value)

In [48]:
predict_df = pd.DataFrame(columns=['가맹점0','가맹점1','가맹점2','가맹점3','가맹점4','가맹점5',
                                   '가맹점6','가맹점7','가맹점8','가맹점9','가맹점10'])
for item in tqdm(y_pred):
    predict_df.loc[len(predict_df)] = item

In [49]:
predict_df['y_test'] = y_test

In [52]:
LIFT20(df_merged, predict_df)

([1.4944736842105264,
  3.550972959344469,
  4.705981424148606,
  3.373337094312073,
  3.8534645825089036,
  4.805892134998514,
  2.7389321257214783,
  3.0369535717450673,
  3.1498619354348847,
  4.37060176098817,
  4.862221373462685],
 3.63115387698867)

In [66]:
df_y_value['MRC_ID_DI'].value_counts()

0     190004
7      32665
6      27347
8      16636
5       7080
1       4175
9       2923
3       1586
10      1378
2        680
4        532
Name: MRC_ID_DI, dtype: int64